In [ ]:
from dateutil.parser import parse
import matplotlib.pyplot as plt
import numpy as np

from src.data_handler import DataHandler

# Set up input file paths
- Give it the measurement folder name and parameters to use
- The measurement folder must be located under the data folder specified in parameters.py (eg. `Z:/Data/20220621_FR0612-F2-2S6_uhv`)

In [ ]:
data_handler = DataHandler(measurement_folder="20220621_FR0612-F2-2S6_uhv")

# Plot all confocal images for a given measurement
- Automatically walk through measurement folders and extract a list of confocal images.
- Each list element is an instance of `MeasurementDataclass` with attributes like `data`, `params` etc.
- For additional info stored in the filename, like height, power etc., call `get_param_from_filename(unit)` with the unit of the data. e.g. to get the power from "20220501_confocal_50mW", the function call will be `get_param_from_filename(unit="mW") => 50`.
- Output folder is automatically created from the output data path `Parameters` and the measurement folder name. For ease of use in presentations/publications etc. images are saved as `.png`, `.pdf`, `.svg` and `.jpg`.

In [ ]:
confocal_list = data_handler.load_measurements_into_dataclass_list(measurement_str="Confocal")

# Set up matplotlib figure
fig, ax = plt.subplots(nrows=len(confocal_list))

# Loop over all confocal images
for idx, confocal in enumerate(confocal_list):
    # Plot each confocal image on a subplot row
    ax[idx].imshow(confocal.data)
    # Extract the power param from the name of file
    power = confocal.get_param_from_filename(unit="mW")
    ax[idx].set_title(f"Laser power = {power}")

# Save output image
data_handler.save_figures(fig, filename="compare_confocals_at different_laser_powers")

## Filter out specific confocal images to plot
Since the confocal images are stored in the list, the required images can just be indexed out

In [ ]:
# Filter out the confocal images from index 10 to 20
filtered_list = confocal_list[10:20]

Alternatively, any of the attributes can be used to filter images

In [ ]:
# Filter based on timestamp, filter all confocal images taken between 10th and 15th May 2022
timestamp_filtered_list = [confocal for confocal in confocal_list if parse("10 May 2022") < confocal.timestamp < parse("15 May 2022")]

# Filter based on laser power, filter all confocal images with laser powers between 25 and 75 mW
param_filtered_list = [confocal for confocal in confocal_list if 25 < confocal.get_param_from_filename(unit="mW") < 75]

# Filter based on image counts, filter all confocal images with maximum counts above 100,000 cps
data_filtered_list = [confocal for confocal in confocal_list if np.max(confocal.data) > int(1e5)]

To reference a specific confocal image or a specific set of confocal images

In [ ]:
# via index
index_confocal = confocal_list[10]

# via timestamp, images taken on 1 May 2022
single_timestamp_confocal = [confocal for confocal in confocal_list if confocal.timestamp.date == parse("1 May 2022").date]

# via set of timestamps
timestamps = [
    parse("1 May 2022 10:30"),
    parse("1 May 2022 10:35"),
    parse("1 June 2022 12:15"),
]
multiple_timestamp_confocals = [confocal for confocal in confocal_list if confocal.timestamp == any(timestamps)]

# Plot all Rabi oscillations for a given measurement and fit each one to an exponentially decaying sinusoid

Rabi oscillations fit to

$$ P_{m_s = 0}(\tau; T_{1\rho}, \omega_R) \propto e^{(-T_{1\rho} \tau)} \sin{(\omega_R \tau)} $$

In [ ]:
data_handler = DataHandler(measurement_folder="20220621_FR0612-F2-2S6_uhv")

rabi_list = data_handler.load_measurements_into_dataclass_list(measurement_str="Rabi")
filtered_rabi_list = [rabi for rabi in rabi_list if
                      parse("10 July 2022 13:30") < rabi.timestamp < parse("10 July 2022 17:30")]

fig, ax = plt.subplots(nrows=len(filtered_rabi_list))

for idx, rabi in enumerate(filtered_rabi_list):
    x, y = rabi.pulsed.measurement.data["t(ns)"], rabi.pulsed.measurement.data["spin_state"]
    fit_x, fit_y = rabi.fit(x=x, y=y, fit_function="sinedoublewithexpdecay")

    ax[idx].plot(x, y, ".")
    ax[idx].plot(fit_x, fit_y, "-")
    ax[idx].set_title(f"Power = {rabi.get_param_from_filename(unit='dBm')}, "
                      f"T1rho = {rabi.fit_result.params['Lifetime']}")

data_handler.save_figures(fig, filename="compare_rabi_oscillations_at different_powers")

## Get a fit report

In [ ]:
rabi = rabi_list[5]

x, y = rabi.pulsed.measurement.data["t(ns)"], rabi.pulsed.measurement.data["spin_state"]

fit_x, fit_y = rabi.fit(x=x, y=y, fit_function="sineexponentialdecay")

rabi.fit_result.fit_report()

# Recompute spin state with different parameters

In [ ]:
rabi = rabi_list[5]

x, timetrace = rabi.pulsed.timetrace.data["t(ns)"], rabi.pulsed.timetrace.data["counts"]

# Analyze using mean, signal from 150 to 400 ns
y_mean = rabi.analyse_mean(timetrace, signal_start=150e-9, signal_end=400e-9)

# Analyze using mean normalized, with normalization from 2000 to 3000 ns
y_mean_norm = rabi.analyse_mean_norm(timetrace, signal_start=150e-9, signal_end=400e-9, norm_start=2000e-9, norm_end=3000e-9)

# Analyze using mean reference, with reference from 2000 to 3000 ns
y_mean_reference = rabi.analyse_mean_reference(timetrace, signal_start=150e-9, signal_end=400e-9, norm_start=2000e-9, norm_end=3000e-9)